Inspired by https://www.ethanrosenthal.com/2020/08/25/optimal-peanut-butter-and-banana-sandwiches/

In [ ]:
import os
import cv2 
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
IMAGE_PATH = os.path.join("..", "images", "test-2.jpg")

# Read image
im = cv2.imread(IMAGE_PATH)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

# Plot
plt.figure(figsize=(16,16))
plt.imshow(im)

In [ ]:
# Convert from RGB to HSV encoding
im_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

plt.figure(figsize=(16,16))
plt.imshow(im_hsv)

In [ ]:
# Extract grayscale image from HSV
h, s, v = cv2.split(im_hsv)

# Find channel with largest contrast
contrasts = [x.std() for x in (h, s, v)]
idx = np.argmax(contrasts)
print(f"Highest contrast is channel {idx}: {contrasts[idx]}")
im_gray = [h, s, v][idx]

plt.figure(figsize=(16,16))
plt.imshow(im_gray, cmap='gray')

## Bilateral Filtering

In [ ]:
# Bilateral filter to reduce noise 
filt = cv2.bilateralFilter(im_gray, 5, 175, 175)

plt.figure(figsize=(16,16))
plt.imshow(filt, cmap='gray')

## Thresholding to eliminate shadows

https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html

In [ ]:
# thresh = cv2.adaptiveThreshold(filt, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 3)

# plt.figure(figsize=(16,16))`
# plt.imshow(thresh, cmap='gray')

In [ ]:
# kernel = np.ones((2, 2), np.uint8)
# thresh_bw = 255 - thresh

# eroded = cv2.erode(thresh_bw, kernel, iterations=2)
# closed = cv2.dilate(eroded, kernel, iterations=2)

# plt.figure(figsize=(16,16))
# plt.imshow(closed, cmap='gray')

## Edge Detection

In [ ]:
# Calculate lower and upper threshold based on mean +/- 2sd
SIGMA = 0.33
median = np.mean(filt)
canny_low = int(max(0, (1 - SIGMA) * median))
canny_high = int(min(255, (1  + SIGMA) * median))
print(canny_low, canny_high)

canny = cv2.Canny(filt, 100, 150)

plt.figure(figsize=(16,16))
plt.imshow(canny, cmap='gray')

## Remove horizonal and vertical lines

https://stackoverflow.com/questions/46274961/removing-horizontal-lines-in-image-opencv-python-matplotlib

In [ ]:
# h_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
# h_line_img = cv2.morphologyEx(canny, cv2.MORPH_OPEN, h_kernel, iterations=1)
# h_line_img = cv2.dilate(h_line_img, h_kernel, iterations=1)
# h_line_cnt, _ = cv2.findContours(h_line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# plt.figure(figsize=(16,16))
# plt.imshow(h_line_img, cmap='gray')

In [ ]:
# v_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
# v_line_img = cv2.morphologyEx(canny, cv2.MORPH_OPEN, v_kernel, iterations=1)
# v_line_img = cv2.dilate(v_line_img, v_kernel, iterations=1)
# v_line_cnt, _ = cv2.findContours(v_line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# plt.figure(figsize=(16,16))
# plt.imshow(v_line_img, cmap='gray')

In [ ]:
# canny_temp = copy.deepcopy(canny)
# LINE_COLOR = (0, 0, 0)
# LINE_THICKNESS = 3
# cv2.drawContours(canny_temp, h_line_cnt, -1, LINE_COLOR, LINE_THICKNESS)
# cv2.drawContours(canny_temp, v_line_cnt, -1, LINE_COLOR, LINE_THICKNESS)

# plt.figure(figsize=(16,16))
# plt.imshow(canny_temp, cmap='gray')

## Thresholding to strengthen canny edges

https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html

In [ ]:
# kernel = np.ones((2, 2), np.uint8)

# closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel, iterations=2)

# plt.figure(figsize=(16,16))
# plt.imshow(closed, cmap='gray')

## Find Contours

In [ ]:
cnt, hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(f"Detected {len(cnt)} contours")

im_temp = copy.deepcopy(im)
LINE_COLOR = (255, 0, 0)
LINE_THICKNESS = 2
cv2.drawContours(im_temp, cnt, -1, LINE_COLOR, LINE_THICKNESS)

plt.figure(figsize=(16,16))
plt.imshow(im_temp)

## Keep Circular Contours

https://www.authentise.com/post/detecting-circular-shapes-using-contours

In [ ]:
circles = []
non_circles = []
MIN_POINTS = 8

# Calculate minimum area as half of mean area of 20 largest contours
TOP_N = 30
cnt_areas = [cv2.contourArea(x) for x in cnt]
cnt_areas_top_n = sorted(cnt_areas)[-TOP_N:]
cnt_areas_median = cnt_areas_top_n[TOP_N // 2]
MIN_AREA = 0.5 * cnt_areas_median
print(f"Median area of top {TOP_N} contours is {cnt_areas_median}")

for i in cnt:
    
    # Approximate polygon closest to each contour
    epsilon = 0.01 * cv2.arcLength(i, True)
    approx = cv2.approxPolyDP(i, epsilon, closed=True)
    
    # Keep only if approximate polygon is like a circle (>8 points + non negligible area)
    area = cv2.contourArea(i)
    if len(approx) > MIN_POINTS and area > MIN_AREA:
        circles.append(i)
    else:
        non_circles.append(i)

print(f"Kept {len(circles)} of {len(cnt)} contours")

# Display results
im_temp = copy.deepcopy(im)
LINE_COLOR = (255, 0, 0)
LINE_THICKNESS = 2
cv2.drawContours(im_temp, circles, -1, LINE_COLOR, LINE_THICKNESS)

# Number contours
for idx, i in enumerate(circles):
    m = cv2.moments(i)
    x = int(m['m10'] / m['m00'])
    y = int(m['m01'] / m['m00'])
    cv2.putText(
        im_temp, 
        text=str(idx + 1), 
        org=(x, y),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=1,
        color=LINE_COLOR,
        thickness=LINE_THICKNESS,
        lineType=cv2.LINE_AA)
        

plt.figure(figsize=(16,16))
plt.imshow(im_temp)